抓取車尾部分並轉換為 ReID 資料集

In [1]:
#抓取車尾部分(label)
import os
import cv2
import numpy as np
from tqdm import tqdm


dataset_path = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling" # r"your_path\trainv2_Frame_Skipping_Handling"
output_folder = os.path.join(dataset_path, 'inside_poly')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 檢查一個點是否在多邊形內的函數
def is_point_in_poly(point, poly):
    x, y = point
    n = len(poly)
    inside = False
    
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y
    
    return inside

# 將bbox的數據轉換為圖像上的坐標函數
def convert_bbox(img_width, img_height, data):
    bb_width = float(data[3]) * img_width
    bb_height = float(data[4]) * img_height
    bb_left = float(data[1]) * img_width - bb_width / 2
    bb_top = float(data[2]) * img_height - bb_height / 2
    return (bb_left, bb_top, bb_left + bb_width, bb_top + bb_height)

camera_polygons = {
    '0': np.array([[750, 0], [800, 0], [1060, 180], [1280, 210], [1280, 720], [680, 720]]),
    '2': np.array([[530, 80], [600, 80], [1060, 180], [1280, 210], [1280, 720], [680, 720]]),
    '4': np.array([[790, 0], [800, 0], [1060, 180], [1280, 210], [1280, 720], [680, 720]]),
    '6': np.array([[400, 150], [800, 150], [1060, 180], [1280, 210], [1280, 720], [400, 720]])
}

# 圖像文件夾路徑和標籤文件夾路徑
image_folder = os.path.join(dataset_path, 'images')
label_folder = os.path.join(dataset_path, 'labels')

# 遍歷圖像文件夾內所有類別文件夾
for class_folder in os.listdir(image_folder):
    # 創建輸出子文件夾
    output_class_folder = os.path.join(output_folder, class_folder)
    if not os.path.exists(output_class_folder):
        os.makedirs(output_class_folder)
    
    # 圖像文件夾路徑和標籤文件夾路徑
    class_image_folder = os.path.join(image_folder, class_folder)
    class_label_folder = os.path.join(label_folder, class_folder)
    
    # 遍歷類別文件夾內所有圖像文件
    for filename in tqdm(os.listdir(class_image_folder)):
        camera_id = filename.split('_')[0]  # 提取相機ID
        if camera_id not in camera_polygons:
            # 如果相機ID不在多邊形列表中，創建空的txt文件
            label_filename = filename.split('.')[0] + '.txt'
            open(os.path.join(output_class_folder, label_filename), 'w').close()
            continue
        
        # 讀取圖像尺寸
        image_path = os.path.join(class_image_folder, filename)
        img = cv2.imread(image_path)
        img_height, img_width = img.shape[:2]
        
        # 讀取標籤並處理需要判斷的圖片
        label_filename = filename.split('.')[0] + '.txt'
        label_path = os.path.join(class_label_folder, label_filename)
        output_label_path = os.path.join(output_class_folder, label_filename)
        
        poly = camera_polygons[camera_id]  # 獲取相機對應的多邊形
        with open(output_label_path, 'w') as output_file, open(label_path, 'r') as label_file:
            labels = label_file.readlines()
            for label in labels:
                data = label.strip().split()
                bbox = convert_bbox(img_width, img_height, data)  # 轉換邊界格式
                bbox_center = ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)

                # 如果邊界框中心在多邊形內，寫入標籤
                if is_point_in_poly(bbox_center, poly):
                    output_file.write(label)



 27%|██▋       | 638/2389 [00:08<00:23, 74.09it/s] 


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import glob
import argparse

from tqdm import tqdm


def convert_bbox(args, data):
    img_height, img_width = args.imgsz
    
    # Normalized bbox to Absoulate
    bb_width = float(data[3]) * img_width
    bb_height = float(data[4]) * img_height
    bb_left = float(data[1]) * img_width - bb_width / 2
    bb_top = float(data[2]) * img_height - bb_height / 2
    track_id = data[5].split('\n')[0]
    
    x1, y1, x2, y2 = bb_left, bb_top, bb_left + bb_width, bb_top + bb_height

    return int(x1), int(y1), int(x2), int(y2), int(track_id)


def make_parser():
    parser = argparse.ArgumentParser("AICUP ReID dataset")

    parser.add_argument("--data_path", default="", help="path to AICUP train data")
    parser.add_argument("--save_path", default="", help="Path to save the MOT-ReID dataset")
    parser.add_argument('--imgsz', type=tuple, default=(720, 1280), help='img size, (height, width)')
    parser.add_argument('--train_ratio', type=float, default=1.0, help='The ratio of the train set when splitting the train set and the test set')

    return parser


def main(args):

    # Create folder for outputs
    save_path = os.path.join(args.save_path, 'AICUP-ReID')
    os.makedirs(save_path, exist_ok=True)

    train_save_path = os.path.join(save_path, 'bounding_box_train')
    os.makedirs(train_save_path, exist_ok=True)
    test_save_path = os.path.join(save_path, 'bounding_box_test')
    os.makedirs(test_save_path, exist_ok=True)
    
    # Split train/test set
    total_files = sorted(glob.glob(os.path.join(args.data_path, 'images', '*')))
    total_count = len(total_files)
    train_count = int(total_count * args.train_ratio)
    train_files = total_files[:train_count]
    
    for time_path in total_files:
        for img_path in tqdm(glob.glob(os.path.join(time_path, '*')), desc=f'convert {time_path}'):
            path_txt = img_path.split(os.sep)
            timestemp, img_name = path_txt[-2], path_txt[-1].split('.')[0]
            frame_id = img_name.split('_')[-1]
            img = cv2.imread(img_path)
            
            with open(os.path.join(args.data_path, 'inside_poly', timestemp, f'{img_name}.txt'), 'r') as gt_file:
                for line in gt_file.readlines():
                    data = line.split(' ')
                    x1, y1, x2, y2, track_id = convert_bbox(args, data)
                    patch = img[y1:y2, x1:x2, :]
                    
                    # Resize patch to 256x256
                    patch_resized = cv2.resize(patch, (256, 256), interpolation=cv2.INTER_CUBIC)
                    
                    # TrackID_TimeStemp_FrameID_acc_data.bmp
                    file_name = f'{str(track_id).zfill(7)}_{timestemp.replace("_", "-")}_{str(frame_id).zfill(7)}_acc_data.bmp'

                    if time_path in train_files:
                        cv2.imwrite(os.path.join(train_save_path, file_name), patch_resized)
                    else:
                        cv2.imwrite(os.path.join(test_save_path, file_name), patch_resized)


if __name__ == "__main__":

    class Args:
        data_path = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling" # r"your_path\trainv2_Frame_Skipping_Handling"
        save_path = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\REID_datasets\Rear_Vehicle"      # r"your_path\REID_datasets\Rear_Vehicle"
        imgsz = (720, 1280)
        train_ratio = 1.0
    
    args = Args()
    main(args)

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from tqdm import tqdm


# 原始圖片和目標圖片的目錄  

# r"your_path\REID_datasets\Rear_Vehicle\AICUP-ReID\bounding_box_train"
# r"your_path\REID_datasets\day-night_card_detection_ESRGAN\AICUP-ReID\bounding_box_train"
source_dir = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\REID_datasets\Rear_Vehicle\AICUP-ReID\bounding_box_train"  
dest_dir = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\REID_datasets\day-night_card_detection_ESRGAN\AICUP-ReID\bounding_box_train"

os.makedirs(dest_dir, exist_ok=True)  # 如果目標資料夾不存在，創建該資料夾

# 定義一個預處理函數將圖片調整為256x256
def resize_image(image):
    return cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)

# 在 ImageDataGenerator 中包含預處理函數
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest',
    featurewise_center=True,
    preprocessing_function=resize_image
)

# 遍歷原始圖片資料夾中的所有圖片
for filename in tqdm(os.listdir(source_dir)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        filebase = os.path.splitext(filename)[0]  # 獲取不帶擴展名的基礎檔名
        img_path = os.path.join(source_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_tensor = np.expand_dims(img, axis=0)

        # 資料增強的迭代生成
        aug_iter = datagen.flow(
            img_tensor,
            batch_size=1,
            save_to_dir=dest_dir,
            save_prefix=filebase + '_aug',  # 在原始檔名後加上 '_aug'
            save_format='bmp'
        )

        # 生成並保存一定數量的增強圖片
        num_augmented_images = 2
        for i in range(num_augmented_images):
            next(aug_iter)

# 資料增強完成
print("資料增強完成，並保存到 '{}' 資料夾。".format(dest_dir))
